# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

#### Task:
1. Get the city data from the .csv file created in the last exercise.
2. Generate a Humidity heat map based on the city data from the above step
3. Filter out a small set of cities that will fit a criterion based on temp, wind speed and cloudiness
4. Get the nearest hotel for these cities
5. plot these cities on the heat map with location markers.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the CSV file as DataFrame
csv_file = os.path.join('.', 'output_data', 'cities.csv')
cities_df = pd.read_csv(csv_file, index_col=False)
cities_df.head()

,City ID,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,-54.80,-68.30,2020-04-11,42.80,56.0,40.0,5.28
1,1,Punta Arenas,CL,-53.15,-70.92,2020-04-11,39.20,86.0,40.0,13.87
2,2,Rio Gallegos,AR,-51.62,-69.22,2020-04-11,42.80,70.0,54.0,6.93
3,3,Bluff,NZ,-46.60,168.33,2020-04-11,70.00,48.0,5.0,5.01
4,4,Kaitangata,NZ,-46.28,169.85,2020-04-11,53.01,85.0,0.0,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Create a Heatmap layer based on Humidity level

#Get the gmaps api key
gmaps.configure(api_key=g_key)

# Get the Humidity level of the each City location along with the Latitude and Longitude
humidity_weight = cities_df["Humidity"].astype(
    float)  # Store Humidity as weights
locations_df = cities_df[["Lat", "Lng"]].astype(
    float)  # Store 'Lat' and 'Lng' into locations DataFrame

# Values to control the size of the figure
figure_layout = {
    'width': '400px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}

# Create the gmaps figure with zoom level
fig = gmaps.figure(figure_layout, center=(20, 10), zoom_level=2)

# Create the Heat Layer for Humidity
heat_layer = gmaps.heatmap_layer(locations_df,
                                 weights=humidity_weight,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=1.5)

# Add the Heat Layer to the figure
fig.add_layer(heat_layer)

# Display the world map with the Humidity Heat layer of the cities globally
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
# 1. Temperature greater than 70 °F
# 2. Wind speed less than 10 mph.
# 3. Cloudiness less than 3 %

hotel_df = cities_df[(cities_df['Max Temp'] > 70)
                     & (cities_df['Wind Speed'] < 10)
                     & (cities_df['Cloudiness'] < 3)]
# Dislpay the Hotel DataFrame based on the above filter Criterion
hotel_df

,City ID,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed
84,84,Karratha,AU,-20.74,116.85,2020-04-11,92.50,29.0,1.0,8.90
145,145,Indramayu,ID,-6.33,108.32,2020-04-11,87.87,61.0,0.0,4.09
228,228,Cabra,PH,13.89,120.05,2020-04-11,83.64,71.0,2.0,3.67
233,233,Pout,SN,14.77,-17.06,2020-04-11,71.60,60.0,0.0,5.82
240,240,Saint-Louis,SN,16.33,-15.00,2020-04-11,73.90,29.0,0.0,4.99
241,241,Goundam,ML,16.41,-3.67,2020-04-11,77.43,26.0,0.0,2.80
251,251,Coahuayana,MX,18.73,-103.68,2020-04-11,75.69,45.0,0.0,3.29
252,252,Araouane,ML,18.90,-3.53,2020-04-11,77.14,14.0,0.0,3.06
257,257,Tombouctou,ML,20.00,-3.00,2020-04-11,77.79,12.0,0.0,5.97
265,265,Makakilo City,US,21.35,-158.09,2020-04-11,80.60,73.0,1.0,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    row_lat = row["Lat"]
    row_lng = row["Lng"]

    # Set the parameters for the web api call
    parameters = {
        "key": g_key  # API Key
        ,"type": "lodging"  # Location Type
        ,"radius": 5000  # Target radius of 5,000 meters
        ,"location":f'{row_lat}, {row_lng}'  # Geographical Coordinates for the location
        ,"rankBy": "distance"  # Rank by distance
    }

    #Call the API and get the response
    response = requests.get(base_url, params=parameters)

    #Get the JSON from the response
    json_response = response.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = json_response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Location"] = json_response["results"][0]["plus_code"]["compound_code"]
        hotel_df.loc[index, "Hotel Lng"] = json_response["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "Hotel Lat"] = json_response["results"][0]["geometry"]["location"]["lat"]
    except:
        print(f"\n[Warning] Hotel found not found in the vicinity of {index}:{row['City']}, {row['Country']} >>>> Skipping.\n"
        )

    print(json.dumps(json_response, indent=4))

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAMU_ixDmG4UQGTq1iHoah3UB3gjQcGTYTOvM1xw5qG5F2WyAQJB2uMOi6AZpRb8IYudgD7JUrje1xO506PXY4iNa1OldsQpsTieFwI8DpXR9MR83Wb3x4n00ICmJ9oXymiyTNxC8sfOuGznnMaUXzto7bfJcgyUQNAxjU_v4qAKchIoUN8bV3VONUGBZCIc183iHoiVyD__RFVVse8wGfuj8EVUMiHtxLO25atZoIbXm0vezfvOkYTYxeiO2Y1KUHzlHvVB0gfJEVUazKHclJfB83yK8RysjkXOOhnJA3qSe2YfEiGpGoGl8jMdQxM47oBcFsIb8dIon-2aT9wobdlDlbvEp-581V1H-odn-b9ySxHOrTFs1i_0YxR6gmXOi5RIQtGNY3dd3aXSKQiqlyDgzPRoUwW14niRHknB5er_YbSse_NYuDPA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.7322255,
                    "lng": 116.8467438
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.7308124197085,
                        "lng": 116.8481896302915
                    },
                    "southwest": {
                        "lat": -20.7335103802915,
                        "lng": 116.8454916

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAADHk9pdTEgTbWdskaSosHQG4B_TzOX4VzhCgUyJEgYgHgDGbuaYDufdam54pAbsFiMMoujtLQIUP16EWuzPyNdZFc-feoVT8vEl9Jh40oqjKtNtT9ZFdME7-A_vUppEbqvwTCYwYGLuvl1xsM-UpN-EhqC0JT9He5tJ2R7ZJ0dmbUNnAev7QKGl_iSCrZDTihTlKw5IPXqxNk40BG-21XqmKx1dIZ5z-PutEQMuWUOka0lA6v8SqDefdiERFNhlHN2CLU30j0QRhoI3fsP6P8hrl-YV1nMGv3ITIH9EJTvgL04X7YBU606YGls6HX6a7diB80FteIbArS8IUGxnvtGB1F-S72W98wTfawd5Z3XMA1lJ2CFGDayXl8t498HDuChIQeOvPciOvU6K89-i-KWtJaRoUARyZyk2vs0uotPpHBF1EBsxVtp0",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -6.3356984,
                    "lng": 108.3325966
                },
                "viewport": {
                    "northeast": {
                        "lat": -6.333956119708497,
                        "lng": 108.3340070802915
                    },
                    "southwest": {
                        "lat": -6.336654080291502,
                        "lng": 108.331309

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 13.889372,
                    "lng": 120.0474777
                },
                "viewport": {
                    "northeast": {
                        "lat": 13.8908060302915,
                        "lng": 120.0488694302915
                    },
                    "southwest": {
                        "lat": 13.8881080697085,
                        "lng": 120.0461714697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "110e22ccf46904f90146459a59b22ba24408083e",
            "name": "Flor Tarcena Villanueva Residence",
            "photos": [
                {
                    "height": 4160,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/1112673171149629


[Warning] Hotel found not found in the vicinity of 257:Tombouctou, ML >>>> Skipping.

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAPom_ZXFc2pzugrkGMctq_W5EJ7KmquwwTUow3ayVcJ8ldvcdV37MR8t8QVmBtK9YDQjlmqkJfb-3F4N6ve4bfhZRU02xluRoQfg5Zs2Ss8aOgSw8VXiMgawiKspuKl9JHG3jOLGglkGeDF_PbUP3aW_IUySQ6YngUofhTV5cMre7xtJ4HY9mmWl5JJ4GA8eE8EMwOW3JestAms8ED5kjuWlnbzR-7QCl1Nax57N280g-jWVLCd_cqDoeezP0b3RTCh9mvDm5ywSg0RiqjbQw65teCGuh8KEP8967jgg6RKtH1IAGfyibc9EDhw8uORz9UDmeYtBgTjr7_E8VGaNCPOdbtmWv1_CTFTioXrqZVJRI8L4lLmY1sZyNDHf73a3FhIQPFDttBKYcR-21vcpymXCWxoUpB7OC06MHMpDgCZVxkdF8TXTHXc",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 21.3317858,
                    "lng": -158.1213696
                },
                "viewport": {
                    "northeast": {
                        "lat": 21.3355554,
                        "lng": -158.11

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 23.2616892,
                    "lng": 68.8301204
                },
                "viewport": {
                    "northeast": {
                        "lat": 23.2629937302915,
                        "lng": 68.8312902802915
                    },
                    "southwest": {
                        "lat": 23.2602957697085,
                        "lng": 68.82859231970849
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "c6a39e47b9493574b018d8ce58eb43d7d6e16095",
            "name": "Hotel Panchvati",
            "photos": [
                {
                    "height": 3480,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/101641197282155367380\">Ritesh Josh

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAIeJ_YUYr8OI2dV9XaiOyFlLjfg8KF8Z4xtt4ml0_LlCjLhvb5o3D5J_o61bZjPSjsrvrlKN1RGfrUETKlBkxyXf78J-IO0ddlFcbmjABpUm8zL3DnbSFNon5H_vqmgadCoYzceRUi7ozfQDcHw_sB4rw0A_HLh8lbz76h5x9uwfJddDJb5PfBo4w570y4n-NHpFMV_4NqML32fmX9vBqT9itS55nA6Wlx0l_fe7L-_nqCF9PQO8L44TYXsU8P3nbosvwYxY05LacajDmtor9tQTWWXzzlApPAKD4VGInXfZ2_Qk41Kg0pbQivQrjAwjXQbjzKLjsiSgI0GwY_IpMVSwaBJOIwZAQDE3ldNXaZuMN9jf_oe2xXxbTj4qrPbDJhIQOUdL4QFDDtlWx1DgY1S-tRoUKOGTkOv9LxSBb0GgzuzGLxIIius",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 23.68421009999999,
                    "lng": 85.5088593
                },
                "viewport": {
                    "northeast": {
                        "lat": 23.6856104802915,
                        "lng": 85.5101081302915
                    },
                    "southwest": {
                        "lat": 23.68291251970849,
                        "lng": 85.50741


[Warning] Hotel found not found in the vicinity of 282:Adrar, DZ >>>> Skipping.

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAF4xhsZ_p-yOZM2Fa76jN8FhQU0wawtXkj4wgtbP0vKduIwMvwgLw7JoIR2SGFIg1Ft_Io7WDRB_TFQHj0q-pXERoVfLfkTljdmsyIDellFTlfrHXBRoIloBPSEO0R4kfdLzhJByf8Ll77-v1Et48nJhxXEMG6F7OqUOohVfPJz6WTWymKEj088KjLCkAivNytRkOdPk_zfjMT9QsLm07_ZyLgHzyGnsvp6mdzkL8Pj313MrOLL0f9kZgaSiZ_ilaFJIr_Rs7iieN5RbucH8ipvdU-f8hJWb1UH24SlX3K-2CPg4QbrHeUZABjODSQCJKAt6SXCN2BUwPkFmrJSL4Gi_h7jM6WOOgh6mHxASqPFnLwM1rqGYubpwAbFVcS315xIQakb4OSk0EheqP00zc8-mkBoUmz5YFYx-kEqn4MDcBUq6DI5i_ww",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 27.9195402,
                    "lng": 96.1707002
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.9209390802915,
                        "lng": 96.17188

In [7]:
hotel_df

,City ID,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Location,Hotel Lng,Hotel Lat
84,84,Karratha,AU,-20.74,116.85,2020-04-11,92.50,29.0,1.0,8.90,ibis Styles Karratha,"7R9W+4M Karratha, Western Australia, Australia",116.846744,-20.732225
145,145,Indramayu,ID,-6.33,108.32,2020-04-11,87.87,61.0,0.0,4.09,Grand Trisula Hotel,"M87M+P2 Karanganyar, Indramayu Regency, West J...",108.332597,-6.335698
228,228,Cabra,PH,13.89,120.05,2020-04-11,83.64,71.0,2.0,3.67,Flor Tarcena Villanueva Residence,"V2QW+PX Lubang, Occidental Mindoro, Philippines",120.047478,13.889372
233,233,Pout,SN,14.77,-17.06,2020-04-11,71.60,60.0,0.0,5.82,chez sada niang,"QWCP+33 Pout, Senegal",-17.064842,14.770173
240,240,Saint-Louis,SN,16.33,-15.00,2020-04-11,73.90,29.0,0.0,4.99,NaN,NaN,NaN,NaN
241,241,Goundam,ML,16.41,-3.67,2020-04-11,77.43,26.0,0.0,2.80,NaN,NaN,NaN,NaN
251,251,Coahuayana,MX,18.73,-103.68,2020-04-11,75.69,45.0,0.0,3.29,Hotel Los Arcos,"P82Q+CJ Coahuayana de Hidalgo, Michoacán, Mexico",-103.660965,18.701049
252,252,Araouane,ML,18.90,-3.53,2020-04-11,77.14,14.0,0.0,3.06,NaN,NaN,NaN,NaN
257,257,Tombouctou,ML,20.00,-3.00,2020-04-11,77.79,12.0,0.0,5.97,NaN,NaN,NaN,NaN
265,265,Makakilo City,US,21.35,-158.09,2020-04-11,80.60,73.0,1.0,5.82,Marriott's Ko Olina Beach Club,"8VJH+PF Kapolei, Hawaii, United States",-158.121370,21.331786


In [8]:
hotel_df.fillna('', inplace=True)
hotel_df

,City ID,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Location,Hotel Lng,Hotel Lat
84,84,Karratha,AU,-20.74,116.85,2020-04-11,92.50,29.0,1.0,8.90,ibis Styles Karratha,"7R9W+4M Karratha, Western Australia, Australia",116.847,-20.7322
145,145,Indramayu,ID,-6.33,108.32,2020-04-11,87.87,61.0,0.0,4.09,Grand Trisula Hotel,"M87M+P2 Karanganyar, Indramayu Regency, West J...",108.333,-6.3357
228,228,Cabra,PH,13.89,120.05,2020-04-11,83.64,71.0,2.0,3.67,Flor Tarcena Villanueva Residence,"V2QW+PX Lubang, Occidental Mindoro, Philippines",120.047,13.8894
233,233,Pout,SN,14.77,-17.06,2020-04-11,71.60,60.0,0.0,5.82,chez sada niang,"QWCP+33 Pout, Senegal",-17.0648,14.7702
240,240,Saint-Louis,SN,16.33,-15.00,2020-04-11,73.90,29.0,0.0,4.99,,,,
241,241,Goundam,ML,16.41,-3.67,2020-04-11,77.43,26.0,0.0,2.80,,,,
251,251,Coahuayana,MX,18.73,-103.68,2020-04-11,75.69,45.0,0.0,3.29,Hotel Los Arcos,"P82Q+CJ Coahuayana de Hidalgo, Michoacán, Mexico",-103.661,18.701
252,252,Araouane,ML,18.90,-3.53,2020-04-11,77.14,14.0,0.0,3.06,,,,
257,257,Tombouctou,ML,20.00,-3.00,2020-04-11,77.79,12.0,0.0,5.97,,,,
265,265,Makakilo City,US,21.35,-158.09,2020-04-11,80.60,73.0,1.0,5.82,Marriott's Ko Olina Beach Club,"8VJH+PF Kapolei, Hawaii, United States",-158.121,21.3318


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

#Create the market layer to be able to plot the markers on the map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

In [10]:
# Add marker layer on top of heat map generated earlier
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))